# Extraction of Henry's Law Constants from Fortran 90 file

Version: 0.3.0

Some code adapted from https://github.com/jtd1g16/HLC_Prediction 

## Importing Libraries

In [1]:
import pandas as pd 
from tqdm import tqdm
from MyFunctions.IdentifierConversion import convertors as conv
import json
from rdkit import RDLogger
from itertools import islice
RDLogger.DisableLog('rdApp.*')
import numpy as np
import cirpy as cp
from rdkit import Chem
import pubchempy as pcp
import requests
import time as t

vers = "2.7.1"

## Reading the file and converting it to a dictionary

In [3]:
def rawToDict(file):
    """ 
    Takes the fortran file and returns a dictionary with the species name as the key and a dictionary with the CAS number,
    the constants, and the types of constants as the value.
    
    Parameters:
    file: file object
        The fortran file containing the Henry's Law constants.
        
    Returns:
    consDict: dictionary
        A dictionary with the species name as the key and a dictionary with the CAS number, the constants, and the types of constants as the value.
    """
    missingTempCounter = 0
    numberOfTemps = 0

    speciesSubset = file.read().split(sep = "! species:") #Split the file into a list to separate each species
    print("Number of species:", len(speciesSubset) - 1) #Print the number of species in the file
    consDict = {} #Initialize the dictionary to store the species name, CAS number, constants, and types of constants
    for lines in tqdm(speciesSubset[1:]): #Reading the data from each species #CHANGE TO [1:] TO READ ALL
        speciesLine = lines.split(sep = "\n")[0] #Separating the line with the name
        speciesName = speciesLine[1:].strip() #Storing name

        inchiLine = lines.split(sep = "\n")[4] #Separating the line with the InChI
        inchiKey = inchiLine.split(sep='! inchikey: ')[1].strip() #Storing InChI

        casLine = lines.split(sep = "\n")[3] #Separating the line with the CAS
        casNum = casLine.split(sep='! casrn:   ')[1].strip() #Storing CAS

        consTypes = []; consList = []; temps = [] #Creating lists for constants
        consLines = lines.split(sep="\n")[5:] #Separating constant lines
        for i in range(len(consLines)): #Iterating through lines 
            if "HscpSI" in consLines[i]: #Checking HscpSI is present
                consTypes.append(consLines[i].split(sep='type: ')[1].split(sep=',')[0]) #Data type
                consList.append(consLines[i].split(sep='=  ')[1][0:8].strip()) #Constant

                temp = consLines[i].split(sep='=  ')[1][14:21].strip()
                if temp == "": #Assuming temperature is 25C if not listed
                    temp = "298.15"
                    missingTempCounter += 1
                temps.append(float(temp)-273.15) #Temperature
                numberOfTemps += 1
            
        consDict[speciesName] = {"Compound" : speciesName, #Constructing the dictionary
                                "InChIKey" : inchiKey,
                                "CAS"       : casNum,
                                "Constants" : consList, 
                                "Types" : consTypes,
                                "Temperature" : temps}

    print("Number of dictionary entries:", len(consDict))

    print("Number of missing temperatures:", missingTempCounter)
    print("Number of temperatures:", numberOfTemps)
    return consDict

try:
    with open(f"../Data/SourceData/{vers}-HLExtractedFull.json", "r") as file: #Checking if json file already exists
        consDict = json.load(file)
    file.close()
    print("Dictionary loaded from file")

except:
    with open("../Data/SourceData/HscpSI.f90", "r") as file: #Reading fortran file and converting to dict
        consDict = rawToDict(file) 
    file.close()

    with open(f"../Data/SourceData/{vers}-HLExtractedFull.json", "w") as file: #Writing dictionary to json file
        json.dump(consDict, file)
    file.close()

    pd.DataFrame(consDict).T.reset_index(drop=True).to_csv(f"../Data/SourceData/{vers}-HLExtractedFull.csv", index=False) #Writing dictionary to csv file

Number of species: 10173


100%|██████████| 10173/10173 [00:00<00:00, 58745.14it/s]

Number of dictionary entries: 10173
Number of missing temperatures: 40862
Number of temperatures: 45221


In [3]:
def casToX(idList1, idList2, idList3, target):
        """ 
        This function converts CAS to canonical SMILES/InChI strings using cirpy.
        
        Parameters:
        casList: list of strings
        target: string

        Returns:
        targVals: list of strings
        """

        # validateTarget(target)

        targVals = []
        for i in tqdm(range(len(idList1)), desc=f"Converting identifier to {target}"):
            id1 = idList1[i]; id2 = idList2[i]; id3 = idList3[i]
            toTry = [id1, id2, id3]
            output = []

            for id in toTry:
                try:
                    if id != "UNKNOWN":
                        val = cp.resolve(id, target)
                        if val != "O" or val == np.nan: #It should never be water for this dataset
                            output.append(val)
                            break
                        else:
                            output.append(np.nan)

                    else: #If the identifier is "UNKNOWN", then the value is np.nan
                        print(f"Error with {id}. Trying next identifier.")
                        output.append(np.nan)
                except:
                    print(f"Error with {id}. Trying next identifier.")
                
            if len(output) == 3 and all([x == np.nan for x in output]):
                print(f"Error with {id1}, {id2}, and {id3}.")
                targVals.append(np.nan)
            else:
                targVals.append(output[0])
        return targVals


def smilesToInChI(smiles, nameList, casList):
    """ 
    This function converts SMILES to InChI strings and adds them to the dataframe. Where an InChI string cannot be found, a nan is added.
    
    Parameters:
    smiles: list of strings
    
    Returns:
    InChI: list of strings
    """
    RDLogger.DisableLog('rdApp.*')
    nErrors = 0; InChI = []
    
    for i in tqdm(range(len(smiles)), desc="Converting SMILES to InChI"): #Iterating through the SMILES strings and converting them to InChI strings
        try:
            mol = Chem.MolFromSmiles(smiles[i]) #Convert to mol object then add inchi to list
            inchival = Chem.MolToInchi(mol)
        except:
            inchival = np.nan

        if type(inchival) != float: #If the InChI is not found, then try other identifiers
            InChI.append(inchival)
        else:
            print("InChI not found for", smiles[i], "trying other identifiers")
            valList = casToX([nameList[i]], [smiles[i]], [casList[i]], "stdinchi")

            if valList[0] != np.nan:
                InChI.append(valList[0])
            else:
                InChI.append(np.nan) #Adding nan and incrementing the error count
                print("No InChI available for", nameList[i])
                nErrors += 1
    print(nErrors, "errors occurred")

    return InChI

## Adding the CAS and InChI

In [4]:
# consDict = dict(islice(consDict.items(), 300)) #Limiting the number of species to 10 for testing

inchiList = [consDict[key]['InChIKey'] for key in consDict.keys()] #Extracting CAS numbers for casToSmiles function
casList = [consDict[key]['CAS'] for key in consDict.keys()] #Extracting CAS numbers for casToSmiles function
nameList = [consDict[key]['Compound'] for key in consDict.keys()] #Extracting CAS numbers for casToSmiles function

try: #Reading SMILES from file if it exists, or generating SMILES from CAS numbers
    with open(f"../Data/Processed/{vers}-HL_SMILES.txt", "r") as infile:
        smiles = eval(infile.read())
    infile.close()
except:
    smiles = casToX(inchiList, casList, nameList, "smiles")
    with open(f"../Data/Processed/{vers}-HL_SMILES.txt", "w") as outfile:
        for i in range(len(smiles)):
            outfile.write(str(smiles[i]) + '\n')
    outfile.close()

print(len(smiles), "SMILES")

try: #Reading InChI from file if it exists, or generating InChI from SMILES
    with open(f"../Data/Processed/{vers}-HL_InChI.txt", "r") as infile:
        InChI = infile.read().split(sep = '\n')
    infile.close()
    InChI = [InChI[i] for i in range(len(InChI) - 1)]
except:
    InChI = smilesToInChI(smiles, nameList, casList)
    with open(f"../Data/Processed/{vers}-HL_InChI.txt", "w") as outfile:
        for i in range(len(InChI)):
            outfile.write(str(InChI[i]) + '\n')
    outfile.close()

print(len(InChI), "InChI")

for key in consDict.keys(): #Adding SMILES and InChI to the dictionary
    consDict[key]['SMILES'] = smiles[inchiList.index(consDict[key]['InChIKey'])]
    consDict[key]['InChI'] = InChI[inchiList.index(consDict[key]['InChIKey'])]

with open(f"../Data/Processed/{vers}-HL_AllDTypes.json", "w") as outfile: #Writing dictionary to json file
    json.dump(consDict, outfile)
outfile.close()

Converting identifier to smiles:   0%|          | 0/10173 [00:00<?, ?it/s]

Converting identifier to smiles:   1%|          | 81/10173 [00:48<2:18:19,  1.22it/s]

Error with RQNWIZPPADIBDY-UHFFFAOYSA-N. Trying next identifier.


Converting identifier to smiles:   1%|          | 94/10173 [00:59<2:04:07,  1.35it/s]

In [ ]:
def averageTemperatureValues(removedTypes, consDict):
    """
    Averages the Henry's Law constants for each compound at different temperatures.

    Parameters:
    removedTypes: list
        A list of types of constants to remove from the dictionary.
    consDict: dictionary
        A dictionary with the species name as the key and a dictionary with the CAS number, the constants, and the types of constants as the value.

    Returns:
    consDict: dictionary
        A dictionary with the species name as the key and a dictionary with the CAS number, the constants, and the types of constants as the value.
    """

    for compound in consDict.keys(): #Iterating through compounds
        df = pd.DataFrame(consDict[compound]) #Creating a dictionary to make it easier to drop dubious values
        for t in removedTypes:
            df = df.drop(df[df["Types"] == t].index) #Removing rows with X as the type
            # print(f"Removed type {t}, resulting shape:", df.shape)

        temperatureDict = {} #Initialising dict
        temperatures = df["Temperature"].values

        for i in range(len(temperatures)): #Iterating through temperatures
            try:
                temperatureDict[temperatures[i]].append(df["Constants"].values[i]) #Adding constant to temperature as list
            except:
                temperatureDict[temperatures[i]] = [df["Constants"].values[i]]

        for key in list(temperatureDict.keys()): #Calculating the mean of the constants at each temperature
            values = temperatureDict[key]
            values = [float(i) for i in values]
            temperatureDict[key] = np.mean(values)

        consDict[compound]["Constants"] = list(temperatureDict.values()) #Updating dictionary
        consDict[compound]["Temperature"] = list(temperatureDict.keys())
        del consDict[compound]["Types"] #Removing types (now redundant)

    return consDict

with open(f"../Data/Processed/{vers}-HL_AllDTypes.json", "r") as outfile: #Writing dictionary to json file
    consDict = json.load(outfile)
outfile.close()

removedTypes = ["C", "X", "?", "E"] # Original types to remove
# removedTypes = ["V", "R", "T", "C", "X", "?", "E"] #Stricter types to remove
consDict = averageTemperatureValues(removedTypes, consDict)

with open(f"../Data/Processed/{vers}-HenrysLaw.json", "w") as outfile: #Saving updated dictionary
    json.dump(consDict, outfile)

## Dataset with expanded lists

In [ ]:
dataset = pd.DataFrame(columns=list(consDict["ozone"].keys())) #Creating a new dataframe to store the data

for key in consDict.keys(): #Creating a dataframe for each species + concatenating
    try:
        df = pd.DataFrame.from_dict(consDict[key])
        dataset = pd.concat([dataset, df], axis=0)
    except: #If there is an error, print the key and the dictionary
        print(key)
        print(consDict[key])

dataset.reset_index(drop=True, inplace=True) #Saving
print(dataset.shape)
dataset.head()
dataset.to_csv(f"../Data/Processed/{vers}-HenrysLaw.csv", index=False)

C:\Users\josh_\AppData\Local\Temp\ipykernel_15804\4219278837.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset = pd.concat([dataset, df], axis=0)


(12255, 7)


In [3]:
dataset.describe()

,Constants,Temperature
count,1.225500e+04,12255.000000
mean,3.031879e+35,1416.951183
std,3.197827e+37,2876.725505
min,8.000000e-14,-6973.150000
25%,7.000000e-03,25.000000
50%,1.000000e+00,25.000000
75%,4.485000e+03,25.000000
max,3.535500e+39,28726.850000


In [4]:
np.sum(dataset.notnull())

c:\Users\josh_\Documents\GitHub\HenrysLaw\.venv\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


Compound       12255
InChIKey       12255
CAS            12255
Constants      12255
Temperature    12255
SMILES          9156
InChI           9578
dtype: int64

In [5]:
dataset["InChI"].value_counts()

InChI
InChI=1S/C7H8/c1-7-5-3-2-4-6-7/h2-6H,1H3                           24
InChI=1S/C6H6/c1-2-4-6-5-3-1/h1-6H                                 21
InChI=1S/C2H3Cl3/c1-2(3,4)5/h1H3                                   20
InChI=1S/C2HCl3/c3-1-2(4)5/h1H                                     20
InChI=1S/C3H6O/c1-3(2)4/h1-2H3                                     18
                                                                   ..
InChI=1S/C9H16O2/c1-3-5-9(11)7-6-8(10)4-2/h3-7H2,1-2H3              1
InChI=1S/C9H12O2/c1-9(2)5-3-6(9)8(11)4-7(5)10/h5-6H,3-4H2,1-2H3     1
InChI=1S/C8H8O2/c1-5-3-8(10)6(2)4-7(5)9/h3-4H,1-2H3                 1
InChI=1S/C8H8O2/c1-2-6-5-7(9)3-4-8(6)10/h3-5H,2H2,1H3               1
InChI=1S/3C2H5.ClH.Pb/c3*1-2;;/h3*1H2,2H3;1H;/q;;;;+1/p-1           1
Name: count, Length: 6995, dtype: int64